In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
n = 4
x_wires = range(n)
y_wires = range(n, 2 * n)

wires = range(0, 2 * n)
dev = qml.device("default.qubit", wires=wires, shots=1)

In [3]:
# calculates y_val - x_val

def tmp_fct():
    qml.QFT(wires=y_wires)
    for j in x_wires:
        for l in range(n - j):
            qml.ControlledPhaseShift(2 * np.pi / (2 ** (n - l - j)), wires=[n - (j+1), 2 * n - (l + 1)])
    qml.adjoint(qml.QFT(wires=y_wires))


@qml.qnode(dev)
def addition(x_val, y_val):
    qml.templates.BasisEmbedding(x_val, wires=x_wires)
    qml.templates.BasisEmbedding(y_val, wires=y_wires)

    qml.QFT(wires=y_wires)
    for j in x_wires:
        for l in range(n - j):
            qml.ControlledPhaseShift(-2 * np.pi / (2 ** (n - l - j)), wires=[n - (j+1), 2 * n - (l + 1)])
    qml.adjoint(qml.QFT(wires=y_wires))
    tmp_fct()
    qml.adjoint(tmp_fct)()

    return qml.sample(wires=wires)

In [4]:
result = addition(1, 4)

x = result[0:n]
y = result[n:2 * n]
print("x = ", x)
print("y = ", y)

x =  [0 0 0 1]
y =  [0 0 1 1]


In [5]:
import pennylane as qml
from pennylane import numpy as np

In [7]:
n_wires = 4
dev = qml.device("default.qubit", wires=n_wires, shots=1)

def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])

@qml.qnode(dev)
def sum(m, k):
    qml.BasisEmbedding(m, wires=range(n_wires)) # m encoding
    qml.QFT(wires=range(n_wires)) # step 1
    add_k_fourier(k, range(n_wires)) # step 2
    qml.adjoint(qml.QFT) (wires=range(n_wires)) # step 3
    return qml.sample()

print(f"The ket representation of the sum of 3 and 4 is {sum(4,3)}")

The ket representation of the sum of 3 and 4 is [0 1 1 1]
